In [109]:
import logging
logging.root.handlers = []  # Jupyter messes up logging so needs a reset
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
from smart_open import smart_open
import pandas as pd
import numpy as np
from numpy import random
import gensim
import nltk
from sklearn.cross_validation import train_test_split
from sklearn import linear_model
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix
import matplotlib.pyplot as plt
from gensim.models import Word2Vec
from sklearn.neighbors import KNeighborsClassifier
from sklearn import linear_model
from nltk.corpus import stopwords
import os, codecs
%matplotlib inline
from bs4 import BeautifulSoup
import re
from nltk.corpus import stopwords
from sklearn import metrics as skmetrics

## Model 1 evaluation

In [110]:
model = gensim.models.Word2Vec.load('/home/bahbbc/workspace/masters-big5/models/tweet50-600.model')
model.init_sims(replace=True)

2017-05-02 22:35:33,950 : INFO : loading Word2Vec object from /home/bahbbc/workspace/masters-big5/models/tweet50-600.model
2017-05-02 22:35:41,585 : INFO : loading wv recursively from /home/bahbbc/workspace/masters-big5/models/tweet50-600.model.wv.* with mmap=None
2017-05-02 22:35:41,586 : INFO : loading syn0 from /home/bahbbc/workspace/masters-big5/models/tweet50-600.model.wv.syn0.npy with mmap=None
2017-05-02 22:35:49,464 : INFO : setting ignored attribute syn0norm to None
2017-05-02 22:35:49,466 : INFO : loading syn1neg from /home/bahbbc/workspace/masters-big5/models/tweet50-600.model.syn1neg.npy with mmap=None
2017-05-02 22:36:45,712 : INFO : setting ignored attribute cum_table to None
2017-05-02 22:36:45,960 : INFO : loaded /home/bahbbc/workspace/masters-big5/models/tweet50-600.model
2017-05-02 22:38:13,188 : INFO : precomputing L2-norms of word weight vectors


In [111]:
print(model.most_similar(positive=['rss', 'haha']))

[(u'hehe', 0.8554530143737793), (u'hahaha', 0.8513627052307129), (u'hahah', 0.8424988389015198), (u'rs', 0.8193700909614563), (u'kk', 0.8075661063194275), (u'rsrs', 0.7939968109130859), (u'hahahah', 0.7900516986846924), (u'kkk', 0.7897874712944031), (u'hehehe', 0.7863951921463013), (u'hahahaha', 0.7844202518463135)]


In [112]:
print(model.most_similar(positive=[':D', ':/']))

[(u':(', 0.795927882194519), (u':p', 0.7728939056396484), (u':s', 0.763850748538971), (u'://', 0.7579135298728943), (u':))', 0.7576685547828674), (u'=/', 0.7541918754577637), (u';/', 0.7447569370269775), (u':S', 0.7422836422920227), (u':P', 0.7380853891372681), (u':\\', 0.7342033386230469)]


In [113]:
print(model.most_similar(positive=['fofo', 'cute']))

[(u'fofinho', 0.7343779802322388), (u'fofo,', 0.6787629127502441), (u'meigo', 0.6691230535507202), (u'lindinho', 0.6611505746841431), (u'kawaii', 0.6287035942077637), (u'fofo!', 0.6275116801261902), (u'fofis', 0.6228358149528503), (u'fofo.', 0.6119152903556824), (u'fofo"', 0.6110981702804565), (u'bonitinho', 0.6054898500442505)]


In [114]:
num_features= 600

In [115]:
df = pd.read_csv('~/personality-normalized-word2vec-norm.csv', encoding='utf-8')
df.shape

(1039, 186)

In [116]:
df.extraversion_m.value_counts()

0    529
1    510
Name: extraversion_m, dtype: int64

In [117]:
train_w2v_data, test_w2v_data = train_test_split(df, test_size=0.3, random_state=42)

In [118]:
def predict(vectorizer, classifier, data):
    data_features = vectorizer.transform(data['formatted_text'])
    predictions = classifier.predict(data_features)
    target = data['extraversion_m']
    evaluate_prediction(predictions, target)

In [119]:
def word_averaging(wv, words):
    all_words, mean = set(), []
    #print words.shape
    for word in words:
        if isinstance(word, np.ndarray):
            mean.append(word)
        elif word in wv.wv.vocab:
            mean.append(wv.wv.syn0norm[wv.wv.vocab[word].index])
            all_words.add(wv.wv.vocab[word].index)
        #print mean

    if not mean:
        logging.warning("cannot compute similarity with no input %s", words)
        # FIXME: remove these examples in pre-processing
        return np.zeros(num_features,)

    mean = gensim.matutils.unitvec(np.array(mean).mean(axis=0)).astype(np.float32)
    return mean

def  word_averaging_list(wv, text_list):
    return np.vstack([word_averaging(wv, review) for review in text_list ])

In [120]:
def w2v_tokenize_text(text):
    tokens = []
    if text is np.nan:
        return []
    for sent in nltk.sent_tokenize(text, language='portuguese'):
        for word in nltk.word_tokenize(sent, language='portuguese'):
            if len(word) < 2:
                continue
            tokens.append(word)
    return tokens

In [121]:
test_tokenized = test_w2v_data.apply(lambda r: w2v_tokenize_text(r['formatted_text']), axis=1).values
train_tokenized = train_w2v_data.apply(lambda r: w2v_tokenize_text(r['formatted_text']), axis=1).values

In [122]:
%%time
X_train_word_average = word_averaging_list(model,train_tokenized)
X_test_word_average = word_averaging_list(model,test_tokenized)

2017-05-02 22:40:58,073 : WARNING : cannot compute similarity with no input []
2017-05-02 22:40:59,165 : WARNING : cannot compute similarity with no input []
2017-05-02 22:40:59,176 : WARNING : cannot compute similarity with no input []
2017-05-02 22:41:00,666 : WARNING : cannot compute similarity with no input []
2017-05-02 22:41:02,420 : WARNING : cannot compute similarity with no input []
2017-05-02 22:41:03,121 : WARNING : cannot compute similarity with no input []
2017-05-02 22:41:04,378 : WARNING : cannot compute similarity with no input []
2017-05-02 22:41:04,579 : WARNING : cannot compute similarity with no input []
2017-05-02 22:41:04,955 : WARNING : cannot compute similarity with no input []
2017-05-02 22:41:05,218 : WARNING : cannot compute similarity with no input []
2017-05-02 22:41:05,371 : WARNING : cannot compute similarity with no input []


CPU times: user 6.98 s, sys: 894 ms, total: 7.87 s
Wall time: 17 s


In [123]:
del model

In [124]:
X_train_word_average.shape

(727, 600)

In [125]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.externals import joblib

force = False
model_trainer = RandomizedSearchCV(
    n_iter=1, 
    estimator=DecisionTreeRegressor(),
    param_distributions={
        "max_features": ["log2"],
        "random_state": [42],
        "criterion":['mse']
    },
    verbose=True,
    refit=True,
    cv=10,
    n_jobs=-1
)

In [126]:
%%time
model_trainer.fit(X_train_word_average, train_w2v_data['extraversion_m'])
model = model_trainer.best_estimator_

Fitting 10 folds for each of 1 candidates, totalling 10 fits


[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:   27.0s finished


CPU times: user 5.74 s, sys: 304 ms, total: 6.04 s
Wall time: 34.3 s


In [127]:
joblib.dump(model, '/home/bahbbc/workspace/masters-big5/extraversion-best-random-tree-model1.pkl')

['/home/bahbbc/workspace/masters-big5/extraversion-best-random-tree-model1.pkl']

In [128]:
yp = model.predict(X_test_word_average)
yt = test_w2v_data['extraversion_m']

In [129]:
pd.DataFrame(
    index=pd.Index([0, 1], name="y_true"),
    columns=pd.Index([0, 1], name="y_pred"),
    data=skmetrics.confusion_matrix(y_true=yt, y_pred=yp)
)

y_pred,0,1
y_true,,
0,100,42
1,78,92


In [130]:
print skmetrics.classification_report(y_true=yt, y_pred=yp)

             precision    recall  f1-score   support

          0       0.56      0.70      0.62       142
          1       0.69      0.54      0.61       170

avg / total       0.63      0.62      0.61       312



In [131]:
accuracy_score(yt, yp)

0.61538461538461542

In [132]:
print '--------- TRAIN -----------------'

--------- TRAIN -----------------


In [133]:
ytp = model.predict(X_train_word_average)
ytt = train_w2v_data['extraversion_m']

In [134]:
print skmetrics.classification_report(y_true=ytt, y_pred=ytp)

             precision    recall  f1-score   support

          0       1.00      1.00      1.00       387
          1       1.00      1.00      1.00       340

avg / total       1.00      1.00      1.00       727



In [135]:
print accuracy_score(ytt, ytp)

0.998624484182
